In [ ]:
!pip install requests

In [ ]:
import requests
import pandas as pd

# Replace YOUR_API_KEY with your actual API key.
API_KEY = "YOUR_API_KEY"



In [ ]:
#https://api.geoapify.com/v1/geocode/search?text=38%20Upper%20Montagu%20Street%2C%20London%20W1H%201LJ%2C%20United%20Kingdom&format=json&apiKey=YOUR_API_KEY
#https://addressvalidation.googleapis.com/v1:validateAddress?key=YOUR_API_KEY
def validate_address(address):
    url = f"https://api.geoapify.com/v1/geocode/search?text={address}&limit=1&apiKey={API_KEY}"
    try:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            if data.get("features"):
                result = data["features"][0]
                latitude = result["geometry"]["coordinates"][1]
                longitude = result["geometry"]["coordinates"][0]
                return "Y", latitude, longitude
            else:
                return "N", None, None
        else:
            return "N", None, None
    except requests.exceptions.RequestException:
        return "N", None, None



In [ ]:
def translate_lat_long_to_address(latitude, longitude):
    url = f"https://api.geoapify.com/v1/geocode/reverse?lat={latitude}&lon={longitude}&apiKey={API_KEY}"
    try:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            if data.get("features"):
                result = data["features"][0]
                return result["properties"]["formatted"]
            else:
                return None
        else:
            return None
    except requests.exceptions.RequestException:
        return None



In [ ]:
def process_excel(input_excel_file, output_excel_file):
    # Read the input CSV file into a pandas DataFrame
    df = pd.read_excel(input_excel_file)

    # Concatenate 'area' and 'city' fields with 'address' field
    df['address'] = df['address'] + ', ' + df['area'] + ', ' + df['city']

    # Apply the validate_address function to each row in the DataFrame and create the 'address_validation', 'latitude', and 'longitude' fields
    df['address_validation'], df['latitude'], df['longitude'] = zip(*df['address'].apply(validate_address))

    # Apply the translate_lat_long_to_address function to each row in the DataFrame and create the 'standard_address' field
    df['standard_address'] = df.apply(lambda row: translate_lat_long_to_address(row['latitude'], row['longitude']), axis=1)

    # Write the updated DataFrame to the output CSV file
    df.to_excel(output_excel_file, index=False)



In [ ]:
# Replace 'YOUR_API_KEY' with the actual API key provided by Geoapify
API_KEY = '0550822313f24722a6b26f779ae2a7f2'
input_excel_file_path = "/content/50records_input.xlsx"  # input data file
output_excel_file_path = "/content/50records_output.xlsx"  # desired path for the output data file

process_excel(input_excel_file_path, output_excel_file_path)


TypeError: ignored